## Fine-tune Llama 3 for Sentiment Analysis

For this hands-on tutorial on fine-tuning a Llama 3 model, I am going to deal with a sentiment analysis on financial and economic information. Sentiment analysis on financial and economic information is highly relevant for businesses for several key reasons, ranging from market insights (gain valuable insights into market trends, investor confidence, and consumer behavior) to risk management (identifying potential reputational risks) to investment decisions (gauging the sentiment of stakeholders, investors, and the general public businesses can assess the potential success of various investment opportunities).

Before the technicalities of fine-tuning a large language model like Llama 3, we have to find the correct dataset to demonstrate the potentialities of fine-tuning.

Particularly within the realm of finance and economic texts, annotated datasets are notably rare, with many being exclusively reserved for proprietary purposes. To address the issue of insufficient training data, scholars from the Aalto University School
of Business introduced in 2014 a set of approximately 5000 sentences. This collection aimed to establish human-annotated benchmarks, serving as a standard for evaluating alternative modeling techniques. The involved annotators (16 people with
adequate background knowledge on financial markets) were instructed to assess the sentences solely from the perspective of an investor, evaluating whether the news potentially holds a positive, negative, or neutral impact on the stock price.

The FinancialPhraseBank dataset is a comprehensive collection that captures the sentiments of financial news headlines from the viewpoint of a retail investor. Comprising two key columns, namely "Sentiment" and "News Headline," the dataset effectively classifies sentiments as either negative, neutral, or positive. This structured dataset serves as a valuable resource for analyzing and understanding the complex dynamics of sentiment in the domain of financial news. It has been used in various studies and research initiatives, since its inception in the work by Malo, P., Sinha, A., Korhonen, P., Wallenius, J., and Takala, P.  "Good debt or bad debt: Detecting semantic orientations in economic texts.", published in the Journal of the Association for Information Science and Technology in 2014.

As a first step, we install the specific libraries necessary to make this example work.

* accelerate is a distributed training library for PyTorch by HuggingFace. It allows you to train your models on multiple GPUs or CPUs in parallel (distributed configurations), which can significantly speed up training in presence of multiple GPUs (we won't use it in our example).
* peft is a Python library by HuggingFace for efficient adaptation of pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters. PEFT methods only fine-tune a small number of (extra) model parameters, thereby greatly decreasing the computational and storage costs.
* bitsandbytes by Tim Dettmers, is a lightweight wrapper around CUDA custom functions, in particular 8-bit optimizers, matrix multiplication (LLM.int8()), and quantization functions. It allows to run models stored in 4-bit precision: while 4-bit bitsandbytes stores weights in 4-bits, the computation still happens in 16 or 32-bit and here any combination can be chosen (float16, bfloat16, float32, and so on).
* transformers is a Python library for natural language processing (NLP). It provides a number of pre-trained models for NLP tasks such as text classification, question answering, and machine translation.
* trl is a full stack library by HuggingFace providing a set of tools to train transformer language models with Reinforcement Learning, from the Supervised Fine-tuning step (SFT), Reward Modeling step (RM) to the Proximal Policy Optimization (PPO) step.

## Installations and imports

In [ ]:
!pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
!pip install -q -U -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U transformers=="4.40.0"
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U peft
!pip install -q -U tensorboard

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
cudf 24.4.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which

The code imports the os module and sets two environment variables:
* CUDA_VISIBLE_DEVICES: This environment variable tells PyTorch which GPUs to use. In this case, the code is setting the environment variable to 0, which means that PyTorch will use the first GPU.
* TOKENIZERS_PARALLELISM: This environment variable tells the Hugging Face Transformers library whether to parallelize the tokenization process. In this case, the code is setting the environment variable to false, which means that the tokenization process will not be parallelized.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

The code import warnings; warnings.filterwarnings("ignore") imports the warnings module and sets the warning filter to ignore. This means that all warnings will be suppressed and will not be displayed. Actually during training there are many warnings that do not prevent the fine-tuning but can be distracting and make you wonder if you are doing the correct things.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In the following cell there are all the other imports for running the notebook

In [ ]:
import numpy as np
import pandas as pd
import os

from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

2024-06-24 13:31:13.360997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 13:31:13.361116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 13:31:13.483540: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
print(f"pytorch version {torch.__version__}")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


Disabling two features in PyTorch related to memory efficiency and speed during operations on the Graphics Processing Unit (GPU) specifically for the scaled dot product attention (SDPA) function.

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

## Preparing the data and the core evaluation functions

The code in the next cell performs the following steps:

1. Reads the input dataset from the all-data.csv file, which is a comma-separated value (CSV) file with two columns: sentiment and text.
2. Splits the dataset into training and test sets, with 300 samples in each set. The split is stratified by sentiment, so that each set contains a representative sample of positive, neutral, and negative sentiments.
3. Shuffles the train data in a replicable order (random_state=10)
4. Transforms the texts contained in the train and test data into prompts to be used by Llama: the train prompts contains the expected answer we want to fine-tune the model with
5. The residual examples not in train or test, for reporting purposes during training (but it won't be used for early stopping), is treated as evaluation data, which is sampled with repetition in order to have a 50/50/50 sample (negative instances are very few, hence they should be repeated)
5. The train and eval data are wrapped by the class from Hugging Face (https://huggingface.co/docs/datasets/index)

This prepares in a single cell train_data, eval_data and test_data datasets to be used in our fine tuning.

In [ ]:
filename1 = "../input/offenseval/offenseval_test.csv"

df_test = pd.read_csv(filename1,
                 encoding="utf-8", encoding_errors="replace")
df_test

,Unnamed: 0,id,tweet,label
0,0,8001,اما انت تقعد طول عمرك لا مبدا ولا راي ثابت يا...,OFF
1,1,8002,بتخاف نسوانك يزعلوا ولا ايه اه يا هلفوت يا ...,OFF
2,2,8003,يا عـسانـى نـبـقى يا عـمري حـبايـب وحـبنـا ي...,NOT
3,3,8004,باقي البيان وينو ما شفنه يا برهان ورينا يا ب...,OFF
4,4,8005,اللهم انت الشافي المعافي اشفيه وجميع مرضى ال...,NOT
...,...,...,...,...
1822,1822,9996,الله لايوفقك يا مهند عسيري يا معوق ولا كان م...,OFF
1823,1823,9997,حبيبي يا يوسف وانت طيب يا صاحبي,NOT
1824,1824,9998,يا بو محمد عشت يا طيب الفالعاشت يمينك يا جزي...,NOT
1825,1825,9999,نا مستني الحلقة بقالي سنتين يا بضان يا ابن الب...,OFF


In [ ]:
filename1 = "../input/offenseval/offenseval_train.csv"

df_train = pd.read_csv(filename1,
                 encoding="utf-8", encoding_errors="replace")
df_train

,Unnamed: 0,tweet,label
0,0,الحمدلله يارب فوز مهم يا زمالك كل الدعم ليكم ي...,NOT
1,1,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه,NOT
2,2,يا رب يا واحد يا حد بحق يوم الاحد ان تهلك بن...,OFF
3,3,هواالحرية يا وجع قلبي عليكي يا اميالله لا يح...,NOT
4,4,يا بكون بحياتك الهم يا ما ما بدي كون,NOT
...,...,...,...
7834,7834,انتو بتوزعوا زيت وسكر فعلا يا عباسيا عباس ما...,NOT
7835,7835,كدا يا عمر متزعلهاش يا حبيبي,NOT
7836,7836,هدا سكن اطفال امارتين من شارقة طالبين فزعتكم ي...,NOT
7837,7837,ومدني بمدد من قوتك واجه به ضعفي وكن لي يا رب...,NOT


In [ ]:
df_train = df_train.drop(df_train.columns[0], axis=1)  # Drop first column then first row
df_train

,tweet,label
0,الحمدلله يارب فوز مهم يا زمالك كل الدعم ليكم ي...,NOT
1,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه,NOT
2,يا رب يا واحد يا حد بحق يوم الاحد ان تهلك بن...,OFF
3,هواالحرية يا وجع قلبي عليكي يا اميالله لا يح...,NOT
4,يا بكون بحياتك الهم يا ما ما بدي كون,NOT
...,...,...
7834,انتو بتوزعوا زيت وسكر فعلا يا عباسيا عباس ما...,NOT
7835,كدا يا عمر متزعلهاش يا حبيبي,NOT
7836,هدا سكن اطفال امارتين من شارقة طالبين فزعتكم ي...,NOT
7837,ومدني بمدد من قوتك واجه به ضعفي وكن لي يا رب...,NOT


In [ ]:
label_map = {'NOT': 'neutral', 'OFF': 'negative'}

# Rename labels using the mapping dictionary
df_train['label'] = df_train['label'].replace(label_map)
df_train = df_train[['label', 'tweet']].rename(columns={'label': 'sentiment', 'tweet': 'text'})
df_train

,sentiment,text
0,neutral,الحمدلله يارب فوز مهم يا زمالك كل الدعم ليكم ي...
1,neutral,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه
2,negative,يا رب يا واحد يا حد بحق يوم الاحد ان تهلك بن...
3,neutral,هواالحرية يا وجع قلبي عليكي يا اميالله لا يح...
4,neutral,يا بكون بحياتك الهم يا ما ما بدي كون
...,...,...
7834,neutral,انتو بتوزعوا زيت وسكر فعلا يا عباسيا عباس ما...
7835,neutral,كدا يا عمر متزعلهاش يا حبيبي
7836,neutral,هدا سكن اطفال امارتين من شارقة طالبين فزعتكم ي...
7837,neutral,ومدني بمدد من قوتك واجه به ضعفي وكن لي يا رب...


In [ ]:

df_test = df_test.drop(df_test.columns[0], axis=1)  # Drop first column then first row
df_test

,id,tweet,label
0,8001,اما انت تقعد طول عمرك لا مبدا ولا راي ثابت يا...,OFF
1,8002,بتخاف نسوانك يزعلوا ولا ايه اه يا هلفوت يا ...,OFF
2,8003,يا عـسانـى نـبـقى يا عـمري حـبايـب وحـبنـا ي...,NOT
3,8004,باقي البيان وينو ما شفنه يا برهان ورينا يا ب...,OFF
4,8005,اللهم انت الشافي المعافي اشفيه وجميع مرضى ال...,NOT
...,...,...,...
1822,9996,الله لايوفقك يا مهند عسيري يا معوق ولا كان م...,OFF
1823,9997,حبيبي يا يوسف وانت طيب يا صاحبي,NOT
1824,9998,يا بو محمد عشت يا طيب الفالعاشت يمينك يا جزي...,NOT
1825,9999,نا مستني الحلقة بقالي سنتين يا بضان يا ابن الب...,OFF


In [ ]:
label_map = {'NOT': 'neutral', 'OFF': 'negative'}

# Rename labels using the mapping dictionary
df_test['label'] = df_test['label'].replace(label_map)
df_test = df_test[['label', 'tweet']].rename(columns={'label': 'sentiment', 'tweet': 'text'})
df_test

,sentiment,text
0,negative,اما انت تقعد طول عمرك لا مبدا ولا راي ثابت يا...
1,negative,بتخاف نسوانك يزعلوا ولا ايه اه يا هلفوت يا ...
2,neutral,يا عـسانـى نـبـقى يا عـمري حـبايـب وحـبنـا ي...
3,negative,باقي البيان وينو ما شفنه يا برهان ورينا يا ب...
4,neutral,اللهم انت الشافي المعافي اشفيه وجميع مرضى ال...
...,...,...
1822,negative,الله لايوفقك يا مهند عسيري يا معوق ولا كان م...
1823,neutral,حبيبي يا يوسف وانت طيب يا صاحبي
1824,neutral,يا بو محمد عشت يا طيب الفالعاشت يمينك يا جزي...
1825,negative,نا مستني الحلقة بقالي سنتين يا بضان يا ابن الب...


In [ ]:
X_test, X_eval = train_test_split(df_test, test_size=0.5, random_state=42)

In [ ]:
X_train = df_train

In [ ]:
def generate_prompt(data_point):
    return f""" Analyze the text enclosed in square brackets,
determine if it contains hate speech or not,
and return the answer as the corresponding label and just the corresponding label nothing else "neutral" or "negative".

            [{data_point["text"]}] = {data_point["sentiment"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""Analyze the text enclosed in square brackets,
determine if it contains hate speech or not,
and return the answer as the corresponding label and just the corresponding label nothing else "neutral" or "negative".
            [{data_point["text"]}] = """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1),
                      columns=["text"])

y_true = X_test.sentiment
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

Next we create a function to evaluate the results from our fine-tuned sentiment model. The function performs the following steps:

1. Maps the sentiment labels to a numerical representation, where 2 represents positive, 1 represents neutral, and 0 represents negative.
2. Calculates the accuracy of the model on the test data.
3. Generates an accuracy report for each sentiment label.
4. Generates a classification report for the model.
5. Generates a confusion matrix for the model.

In [ ]:
def evaluate(y_true, y_pred):
    labels = [ 'neutral', 'negative']
    mapping = { 'neutral': 1,'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
     # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
model_name = "../input/llama-3/transformers/8b-chat-hf/1"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 129
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In the next cell, we set a function for predicting the sentiment of a news headline using the Llama-3 language model. The function takes three arguments:

test: A Pandas DataFrame containing the news headlines to be predicted.
model: The pre-trained Llama-3 language model.
tokenizer: The tokenizer for the Llama-3 language model.

The function works as follows:

1. For each news headline in the test DataFrame:
    * Create a prompt for the language model, which asks it to analyze the sentiment of the news headline and return the corresponding sentiment label.
    * Use the pipeline() function from the Hugging Face Transformers library to generate text from the language model, using the prompt.
    * Extract the predicted sentiment label from the generated text.
    * Append the predicted sentiment label to the y_pred list.
2. Return the y_pred list.

The pipeline() function from the Hugging Face Transformers library is used to generate text from the language model. The task argument specifies that the task is text generation. The model and tokenizer arguments specify the pre-trained Llama-2 language model and the tokenizer for the language model. The max_new_tokens argument specifies the maximum number of new tokens to generate. The temperature argument controls the randomness of the generated text. A lower temperature will produce more predictable text, while a higher temperature will produce more creative and unexpected text.

The if statement checks if the generated text contains the word "positive". If it does, then the predicted sentiment label is "positive". Otherwise, the if statement checks if the generated text contains the word "negative". If it does, then the predicted sentiment label is "negative". Otherwise, the if statement checks if the generated text contains the word "neutral". If it does, then the predicted sentiment label is "neutral.

In [ ]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:

            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

## Fine-tuning

In the next cell we set everything ready for the fine-tuning. We configures and initializes a Simple Fine-tuning Trainer (SFTTrainer) for training a large language model using the Parameter-Efficient Fine-Tuning (PEFT) method, which should save time as it operates on a reduced number of parameters compared to the model's overall size. The PEFT method focuses on refining a limited set of (additional) model parameters, while keeping the majority of the pre-trained LLM parameters fixed. This significantly reduces both computational and storage expenses. Additionally, this strategy addresses the challenge of catastrophic forgetting, which often occurs during the complete fine-tuning of LLMs.

PEFTConfig:

The peft_config object specifies the parameters for PEFT. The following are some of the most important parameters:

* lora_alpha: The learning rate for the LoRA update matrices.
* lora_dropout: The dropout probability for the LoRA update matrices.
* r: The rank of the LoRA update matrices.
* bias: The type of bias to use. The possible values are none, additive, and learned.
* task_type: The type of task that the model is being trained for. The possible values are CAUSAL_LM and MASKED_LM.

TrainingArguments:

The training_arguments object specifies the parameters for training the model. The following are some of the most important parameters:

* output_dir: The directory where the training logs and checkpoints will be saved.
* num_train_epochs: The number of epochs to train the model for.
* per_device_train_batch_size: The number of samples in each batch on each device.
* gradient_accumulation_steps: The number of batches to accumulate gradients before updating the model parameters.
* optim: The optimizer to use for training the model.
* save_steps: The number of steps after which to save a checkpoint.
* logging_steps: The number of steps after which to log the training metrics.
* learning_rate: The learning rate for the optimizer.
* weight_decay: The weight decay parameter for the optimizer.
* fp16: Whether to use 16-bit floating-point precision.
* bf16: Whether to use BFloat16 precision.
* max_grad_norm: The maximum gradient norm.
* max_steps: The maximum number of steps to train the model for.
* warmup_ratio: The proportion of the training steps to use for warming up the learning rate.
* group_by_length: Whether to group the training samples by length.
* lr_scheduler_type: The type of learning rate scheduler to use.
* report_to: The tools to report the training metrics to.
* evaluation_strategy: The strategy for evaluating the model during training.

SFTTrainer:

The SFTTrainer is a custom trainer class from the TRL library. It is used to train large language models (also using the PEFT method).

The SFTTrainer object is initialized with the following arguments:

* model: The model to be trained.
* train_dataset: The training dataset.
* eval_dataset: The evaluation dataset.
* peft_config: The PEFT configuration.
* dataset_text_field: The name of the text field in the dataset.
* tokenizer: The tokenizer to use.
* args: The training arguments.
* packing: Whether to pack the training samples.
* max_seq_length: The maximum sequence length.

Once the SFTTrainer object is initialized, it can be used to train the model by calling the train() method

In [ ]:
output_dir="trained_weigths"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=False,             # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)
tokenizer.pad_token = tokenizer.eos_token
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/7839 [00:00<?, ? examples/s]

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

The following code will train the model using the trainer.train() method and then save the trained model to the trained-model directory. Using The standard GPU P100 offered by Kaggle, the training should be quite fast.

In [ ]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.482500,1.622490


TrainOutput(global_step=979, training_loss=1.584071455498638, metrics={'train_runtime': 8995.0082, 'train_samples_per_second': 0.871, 'train_steps_per_second': 0.109, 'total_flos': 2.936095551307776e+16, 'train_loss': 1.584071455498638, 'epoch': 0.9991070289577753})

The model and the tokenizer are saved to disk for later usage.

In [ ]:
# Save trained model and tokenizer
trainer.save_model()
tokenizer.save_pretrained(output_dir)

Afterwards, loading the TensorBoard extension and start TensorBoard, pointing to the logs/runs directory, which is assumed to contain the training logs and checkpoints for your model, will allow you to understand how the models fits during the training.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/runs

In [ ]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 913/913 [08:03<00:00,  1.89it/s]

Accuracy: 0.929
Accuracy for label 0: 0.800
Accuracy for label 1: 0.960

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.80      0.82       180
           1       0.95      0.96      0.96       733

    accuracy                           0.93       913
   macro avg       0.89      0.88      0.89       913
weighted avg       0.93      0.93      0.93       913


Confusion Matrix:
[[144  36]
 [ 29 704]]
